# nextstep demonstration

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## 1. oil price

In [2]:
from nextstep.getData.oil import oil_price
oil_price()

## 2. weather data

In [3]:
from nextstep.getData.weather import get_weather_data
user_config = {'frequency' : 1,
               'start_date' : '01-Jan-2010',
               'end_date' : '31-Jan-2010',
               'api_key' : '2c9e967a17ba475087893244201503',
               'location_list' : ['singapore'],
               'location_label' : False}
get_weather_data(user_config)



Retrieving weather data for singapore


Currently retrieving data for singapore: from 2010-01-01 to 2010-01-31
Time elapsed (hh:mm:ss.ms) 0:00:04.632072
Currently retrieving data for singapore: from 2010-01-01 to 2010-01-31
Time elapsed (hh:mm:ss.ms) 0:00:09.270552


export singapore completed!




## 3. feature selection

In [7]:
from nextstep.feature_selection import majority_voting

In [8]:
df = pd.read_csv('feature_mart_demo.csv')
df.head(3)

,USEP,TRADING_DATE,general_demand,primary_demand_requirement,contigency_demand_requirement,regulation_demand_requirement,primary_demand_price,contigency_demand_price,regulation_demand_price,public_holidays,...,PERIOD_39,PERIOD_40,PERIOD_41,PERIOD_42,PERIOD_43,PERIOD_44,PERIOD_45,PERIOD_46,PERIOD_47,PERIOD_48
0,97.88,2019-06-01,0.529158,0.188993,0.688221,0.614216,0.000005,0.002736,0.004400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,95.99,2019-06-01,0.496337,0.201393,0.688221,0.495411,0.000005,0.001927,0.004233,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,93.94,2019-06-01,0.455330,0.181321,0.638475,0.428513,0.000005,0.002746,0.003333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
y = df['USEP']
X = df.drop(columns=['TRADING_DATE','USEP'])

In [10]:
# majority voting
majority_voting(X, y, num_features=15)

,Feature,kbest_f_regression,kbest_mutual_info_regression,rfe_linear_regression,rfe_svr,regression_tree,lgb_regressor,Total
1,contigency_demand_price,True,True,False,True,True,True,5
2,regulation_demand_price,True,False,False,True,True,True,4
3,general_demand,True,False,False,True,True,True,4
4,wti,False,True,False,False,False,True,2
5,demand_clusters_peak,True,False,True,False,False,False,2
...,...,...,...,...,...,...,...,...
237,CON_max_qty,False,False,False,False,False,False,0
238,CON_max_price,False,False,False,False,False,False,0
239,CON_fifty_supply,False,False,False,False,False,False,0
240,CCPOutage,False,False,False,False,False,False,0


In [ ]:
# tentative
# fs.get_features_from_SelectKBest(f_regression, X, y, num_features=15)

## 4. model

In [11]:
from nextstep.model.random_forest import random_forest

random_forest_shell = random_forest({
                                'label_column' : 'USEP',
                                'train_size' : 0.7,
                                'seed' : 33,
                                'n_estimators' : 25,
                                'bootstrap' : True,
                                'criterion' : 'mse',
                                'max_features' : 'sqrt'})

rf = random_forest_shell.build_model(df.drop(columns=['TRADING_DATE']))

Building random forest model.
Evaluating random forest performance.
MAE : 4.483452044127191
MSE : 238.17569035992207
RMSE : 15.432941727354576


## 5. hyper-paramter tuning